In [1]:
import pandas as pd
import requests
from tqdm.notebook import tqdm

In [2]:
PATH_TO_DATASETS = "../Datasets/"

# GREEN SPACES

This dataset describes an overview of public green, except for solitary trees. This data collection includes location, type of vegetation, and management information.

In [3]:
df = pd.read_csv(f'{PATH_TO_DATASETS}openbaar-groen0.csv', sep=';')

In [28]:
df[['LAT', 'LON']] = df['geo_point_2d'].str.split(',', expand=True)
df['LAT'] = pd.to_numeric(df['LAT'], errors='coerce')
df['LON'] = pd.to_numeric(df['LON'], errors='coerce')

In [ ]:
#df.to_csv(f'{PATH_TO_DATASETS}green-spaces.csv', index=False)

In [32]:
df = pd.read_csv(f'{PATH_TO_DATASETS}green-spaces.csv')

In [33]:
df.drop(columns=['geo_point_2d', 'geo_shape', 'TECHN_KWAL', 'STREET'], inplace=True)

In [ ]:
#df.to_csv(f'{PATH_TO_DATASETS}green-spaces.csv', index=False)

# AIR POLLUTION

In [49]:
df = pd.read_csv(f'{PATH_TO_DATASETS}merged_air_pollution_data.csv')

In [50]:
df

,Sensor,Time_UTC,Time_Local,Lat,Lon,PM1,PM2.5,PM10,NO2
0,I02,2021-01-01 00:00:00,2021-01-01 01:00:00,"51,4379","5,3582","27,78","33,11","37,81",24.0
1,I02,2021-01-02 00:00:00,2021-01-02 01:00:00,"51,4379","5,3582","31,84","45,09","51,59",19.0
2,I02,2021-01-03 00:00:00,2021-01-03 01:00:00,"51,4379","5,3582","23,72","34,22","38,46",13.0
3,I02,2021-01-04 00:00:00,2021-01-04 01:00:00,"51,4379","5,3582","5,35","5,97","7,94",9.0
4,I02,2021-01-05 00:00:00,2021-01-05 01:00:00,"51,4379","5,3581","21,11","23,40","24,02",10.0
...,...,...,...,...,...,...,...,...,...
68735,NaN,2025-03-16 00:00:00,2025-03-16 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN
68736,NaN,2025-03-17 00:00:00,2025-03-17 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN
68737,NaN,2025-03-18 00:00:00,2025-03-18 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN
68738,NaN,2025-03-19 00:00:00,2025-03-19 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
cols_to_convert = ['Lat', 'Lon', 'PM1', 'PM2.5', 'PM10']
for col in cols_to_convert:
    df[col] = df[col].str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [52]:
df['Time_Local'] = pd.to_datetime(df['Time_Local'], errors='coerce')

In [54]:
df.drop(columns=['Time_UTC', 'Lat', 'Lon'], inplace=True)

In [55]:
essential_cols = ['Sensor',  'PM1', 'PM2.5', 'PM10']
df.dropna(subset=essential_cols, inplace=True)

In [56]:
df.reset_index(drop=True, inplace=True)

In [57]:
grouped_df = df.groupby('Sensor').agg({
    'PM1': ['mean', 'median', 'std', 'min', 'max'],
    'PM2.5': ['mean', 'median', 'std', 'min', 'max'],
    'PM10': ['mean', 'median', 'std', 'min', 'max']
}).reset_index()

In [58]:
yearly_stats = []
for sensor, group in df.groupby('Sensor'):
    group = group.set_index('Time_Local')
    numeric_cols = group.select_dtypes(include='number').columns
    stats = group[numeric_cols].resample('YE').agg(['mean', 'median', 'std', 'min', 'max'])
    stats.columns = ['_'.join(col) for col in stats.columns]
    stats['Sensor'] = sensor
    yearly_stats.append(stats.reset_index())

yearly_stats_df = pd.concat(yearly_stats, ignore_index=True)

In [59]:
yearly_stats_df

,Time_Local,PM1_mean,PM1_median,PM1_std,PM1_min,PM1_max,PM2.5_mean,PM2.5_median,PM2.5_std,PM2.5_min,...,PM10_median,PM10_std,PM10_min,PM10_max,NO2_mean,NO2_median,NO2_std,NO2_min,NO2_max,Sensor
0,2022-12-31,9.156860,6.55,6.842175,3.03,33.37,12.257101,9.020,9.158179,4.09,...,19.81,10.688937,10.12,74.81,23.207729,22.0,10.183288,3.0,53.0,I01
1,2023-12-31,6.799699,5.17,5.052150,0.48,24.58,9.129397,7.180,6.153821,0.67,...,15.26,7.153786,3.49,45.41,12.167123,11.0,6.089715,2.0,38.0,I01
2,2024-12-31,7.021284,5.20,5.543151,1.03,33.19,9.285847,7.175,6.581801,1.60,...,14.09,7.434933,5.54,49.00,12.410959,11.0,6.382143,3.0,44.0,I01
3,2025-12-31,13.545949,10.49,9.330134,2.09,34.19,17.053671,14.020,11.746306,3.29,...,19.40,11.578704,6.97,58.18,20.620253,17.0,10.672639,5.0,49.0,I01
4,2021-12-31,12.842493,10.49,7.789723,0.77,35.42,15.666274,13.140,9.193143,1.41,...,18.74,9.643072,5.22,54.53,23.760870,20.0,13.121306,5.0,68.0,I02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,2025-12-31,13.298101,13.70,7.254333,0.95,24.36,17.048101,16.350,10.064476,1.78,...,21.80,12.222790,6.39,57.41,25.354430,27.0,9.046940,6.0,45.0,I62
235,2024-12-31,12.356667,8.52,8.100148,4.21,32.14,14.527333,12.370,8.657447,5.72,...,17.72,7.554285,8.76,41.05,12.000000,10.0,6.813851,5.0,30.0,I63
236,2025-12-31,17.548481,14.55,11.142634,2.81,40.56,20.782658,17.420,13.694785,3.81,...,23.14,13.689710,7.57,66.02,22.645570,21.0,11.518144,4.0,58.0,I63
237,2024-12-31,10.151333,7.63,6.092234,2.76,22.26,12.236667,9.580,6.924522,4.00,...,15.70,6.401919,8.58,35.10,16.200000,18.0,6.635403,7.0,27.0,I64


In [ ]:
#yearly_stats_df.to_csv(f'{PATH_TO_DATASETS}merged_air_pollution_data_clean.csv', index=False)